# Advent of Code 2021 - PowerShell

## Day 3 - Binary Diagnostic

[Puzzle Link](https://adventofcode.com/2021/day/3)

### Input

In [ ]:
$UseSampleData = $false
$DataFile = $UseSampleData ? '..\PuzzleInput\day3_sample.txt' : '..\PuzzleInput\day3.txt'

[string[]]$ReportData = (Get-Content -Path $DataFile -Raw).Split([System.Environment]::NewLine)
$DataBits = $ReportData[0].Length

### Puzzle 1

What is the power consumption of the submarine?

In [ ]:
$Gamma = $Epsilon = [string]::Empty
for ($BitPosition = 0; $BitPosition -lt $DataBits; $BitPosition++) {
    $BitOn = $ReportData.Where{[string][char[]]$_[$BitPosition] -eq '1'}.Count
    $BitOff = $ReportData.Count - $BitOn
    $Gamma += $BitOn -ge $BitOff ? '1' : '0'
    $Epsilon += $BitOff -le $BitOn ? '0' : '1'
}

'Gamma: {0}, Epsilon: {1}' -f $Gamma,$Epsilon
'Power Consumption: {0}' -f ([Convert]::ToInt32($Gamma,2) * [Convert]::ToInt32($Epsilon,2))

# sample data: 198
# answer: 693486

Gamma: 000010110001, Epsilon: 111101001110
Power Consumption: 693486


### Puzzle 2

What is the life support rating of the submarine?

In [ ]:
function Get-Sum {
    param($Numbers)
    ($Numbers | Measure-Object -Sum).Sum
}

function Get-AtmosphereRating {
    param(
        $Readings,
        [switch]$ScrubberData,
        [int]$DataBits
    )
    $Position = $DataBits - 1
    $Data = $Readings.Clone()
    while ($Position -ge 0 -and $Data.Count -gt 1) {        
        $Ones = foreach ($Bit in $Data) { ($Bit -band (1 -shl $Position)) -shr $Position }
        $Ones = Get-Sum -Numbers $Ones        
        $Zeros = $Data.Count - $Ones
        if ($ScrubberData) {
            $Data = $Zeros -gt $Ones ?
            ($Data | Where-Object { $_ -band (1 -shl $Position) }) :
            ($Data | Where-Object { -Not ( $_ -band (1 -shl $Position) )})
        } else {
            $Data = $Zeros -gt $Ones ?
            ($Data | Where-Object { -Not ( $_ -band (1 -shl $Position) )}) :
            ($Data | Where-Object { $_ -band (1 -shl $Position) })
        }
        $Position--
   }
   $Data[0]
}

$Data = $ReportData | Foreach-Object {[Convert]::ToInt32($_,2)}

$O2Data = $Data.Clone()
$CO2Data = $Data.Clone()

$O2Generator =  Get-AtmosphereRating -Readings $O2Data -DataBits $DataBits
$CO2Scrubber =  Get-AtmosphereRating -Readings $CO2Data -DataBits $DataBits -ScrubberData

'Oxygen Generator: {0}, CO2 Scrubber: {1}' -f $O2Generator,$CO2Scrubber
'Life Support Rating: {0}' -f ($O2Generator * $CO2Scrubber)

# sample answer: 230
# answer: 3379326

Oxygen Generator: 933, CO2 Scrubber: 3622
Life Support Rating: 3379326
